# **Модуль Б**. Разработка модели машинного обучения

## Импортирование библиотек

In [107]:
# модули для работы с моделью
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

import pandas as pd

from IPython.display import Audio

from Model.tokenizer import Tokenizer

import numpy as np

## Загрузка данных

In [108]:
# путь к обработанным данным
df_path = '../Module1/Dataset/dataset.h5'
# загружаем датасета
df = pd.read_hdf(df_path, key='df')
df.head()

,path,sentence,record_duration,sampling_rate,array,text_length,snr,rms_dB,tokens,token_ids,mel_spec
0,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,"Абай был не только талантливым поэтом, но и уч...",5.04,16000,"[-1.0186341e-10, 6.91216e-11, -8.0035534e-11, ...",51,29.652618,-21.792862,"[абай, _, был, _, не, _, только, _, талантливы...","[5165, 3734, 5298, 3734, 2808, 3734, 932, 3734...","[[-13.040601, -10.495326, -7.282387, -6.365622..."
1,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,Гибкая кошка легко взбирается на высокое дерево.,5.65,16000,"[2.910383e-11, -1.1641532e-10, 1.9645086e-10, ...",48,27.797586,-23.426945,"[гибкая, _, кошка, _, легко, _, взбирается, _,...","[3706, 3734, 2332, 3734, 1692, 3734, 4548, 373...","[[-3.2210479, -4.417929, -6.569694, -5.2323184..."
2,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,Сделать нужно гораздо больше.,3.64,16000,"[-1.3096724e-10, -5.820766e-11, -1.4551915e-10...",29,27.732599,-24.547728,"[сделать, _, нужно, _, гораздо, _, больше]","[4891, 3734, 6461, 3734, 1353, 3734, 395, 6674...","[[-3.1968756, -3.3615067, -3.878891, -5.443349..."
3,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,"Эти ноты, сливаясь воедино, образуют симфонию,...",6.62,16000,"[8.731149e-11, -6.91216e-11, 0.0, -8.0035534e-...",77,23.927074,-25.052473,"[эти, _, ноты, _, сливаясь, _, воедино, _, обр...","[243, 3734, 5291, 3734, 3835, 3734, 3604, 3734...","[[-12.80517, -11.202806, -9.942759, -6.5413194..."
4,Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14...,Квадрат гипотенузы равен сумме квадратов катетов.,5.83,16000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",49,35.466934,-17.979994,"[квадрат, _, гипотенузы, _, равен, _, сумме, _...","[5016, 3734, 3892, 3734, 4864, 3734, 4855, 373...","[[-5.968595, -4.914544, -4.8844056, -4.73808, ..."


In [109]:
text_example = df['sentence'][42]
text_example

'Европейский союз будет оставаться наблюдателем в Генеральной Ассамблее.'

**Класс с моделью**

In [110]:
import math

In [111]:
class TTSTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=256, n_mels=80, time_steps=429):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        
        # Encoder
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead=4),
            num_layers=3
        )
        
        # Decoder: предсказывает только `n_mels`
        self.decoder = nn.Sequential(
            nn.Linear(d_model, 512),
            nn.ReLU(),
            nn.Linear(512, n_mels)  # (batch, seq_len, n_mels)
        )

        # Исправленный Conv1d: адаптируем `seq_len → time_steps`
        self.time_projection = nn.Conv1d(
            in_channels=n_mels, 
            out_channels=n_mels, 
            kernel_size=3, 
            stride=1, 
            padding=1
        )

        self.n_mels = n_mels
        self.time_steps = time_steps

    def forward(self, tokens):
        # print('tokens', tokens)
        x = self.embedding(tokens)  # (batch, seq_len, d_model)
        # print('x1', x.shape)
        x = self.encoder(x)         # (batch, seq_len, d_model)
        # print('x2', x.shape)
        x = self.decoder(x)         # (batch, seq_len, n_mels)
        # print('x3', x.shape)
        
        # Транспонируем для Conv1D: (batch, seq_len, n_mels) → (batch, n_mels, seq_len)
        x = x.permute(0, 2, 1)

        # Используем Conv1D для приведения seq_len к time_steps
        x = F.interpolate(x, size=self.time_steps, mode="linear", align_corners=True)

        # Возвращаем обратно: (batch, n_mels, time_steps) → (batch, time_steps, n_mels)
        x = x.permute(0, 2, 1)

        return x


In [112]:
class TTSDataset(Dataset):
    def __init__(self, tokenized_texts, mel_spectrograms, 
                 pad_token_id=0, mel_pad_value=-100):
        self.tokenized_texts = tokenized_texts
        self.mel_spectrograms = mel_spectrograms
        self.pad_token_id = pad_token_id
        self.mel_pad_value = mel_pad_value
        
        self.n_mels = mel_spectrograms[0].shape[1]

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        return (
            torch.LongTensor(self.tokenized_texts[idx]),
            torch.FloatTensor(self.mel_spectrograms[idx])
        )

In [113]:
tokenized_texts = df['token_ids']
mel_spectrograms = df['mel_spec']


In [114]:
# Создаем DataLoader
dataset = TTSDataset(tokenized_texts, mel_spectrograms)
dataloader = DataLoader(dataset, batch_size=32)

In [115]:
print(dataset.__getitem__(0)[0])
print(dataset.__getitem__(0)[0].shape)
print(dataset.__getitem__(0)[1])
print(dataset.__getitem__(0)[1].shape)

tensor([5165, 3734, 5298, 3734, 2808, 3734,  932, 3734,  460, 3734, 5910, 3734,
        4008, 3734, 4773, 3734, 1197, 6674, 6674, 6674, 6674, 6674, 6674, 6674,
        6674, 6674, 6674])
torch.Size([27])
tensor([[-13.0406, -10.4953,  -7.2824,  ...,   0.0000,   0.0000,   0.0000],
        [-11.2965, -10.0045,  -5.7739,  ...,   0.0000,   0.0000,   0.0000],
        [-11.3287,  -7.0065,  -3.9856,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [-13.5660, -12.6440, -10.5475,  ...,   0.0000,   0.0000,   0.0000],
        [-12.8161, -10.6422,  -9.5745,  ...,   0.0000,   0.0000,   0.0000],
        [-13.2538, -10.8003,  -9.7605,  ...,   0.0000,   0.0000,   0.0000]])
torch.Size([80, 429])


In [116]:
# Проверяем один пример
tokens, mel = dataset[0]
print("Пример 0:")
print(f"Токены: {tokens} (длина: {len(tokens)})")
print(f"Спектрограмма: {mel.shape}")

Пример 0:
Токены: tensor([5165, 3734, 5298, 3734, 2808, 3734,  932, 3734,  460, 3734, 5910, 3734,
        4008, 3734, 4773, 3734, 1197, 6674, 6674, 6674, 6674, 6674, 6674, 6674,
        6674, 6674, 6674]) (длина: 27)
Спектрограмма: torch.Size([80, 429])


In [117]:
# Проверяем батч
for batch_idx, (tokens_batch, mels_batch) in enumerate(dataloader):
    print(f"\nБатч {batch_idx}:")
    print(f"Токены: {tokens_batch.shape}")
    print(f"Спектрограммы: {mels_batch.shape}")
    break


Батч 0:
Токены: torch.Size([32, 27])
Спектрограммы: torch.Size([32, 80, 429])


In [118]:
tokenizer = Tokenizer()

/home/user/Chemp/Speech-synthesis/Module2


In [119]:
print(f"Размер словаря: {len(tokenizer.text_to_ids_voc)}")

Размер словаря: 6675


In [120]:
import torch.optim as optim

# Параметры
vocab_size = 6675  # размер словаря
n_mels = 80
time_steps = 429

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [121]:
# Создание модели
model = TTSTransformer(vocab_size=vocab_size, 
                       d_model=256, 
                       n_mels=n_mels,
                       time_steps=time_steps
                       )
model.to(device)

/home/user/Chemp/.venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


TTSTransformer(
  (embedding): Embedding(6675, 256)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=80, bias=True)
  )
  (time_projection): Conv1d(80, 80, kernel_size=(3,), stride=(1,), paddin

In [122]:
# Оптимизатор
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [123]:
# Функция потерь (MSE, так как предсказываем спектрограммы)
criterion = nn.MSELoss()

In [124]:
torch.cuda.empty_cache()

In [152]:
from tqdm import tqdm  # Импортируем tqdm

num_epochs = 1000  # Количество эпох
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Прогресс-бар для всех батчей в эпохе
    with tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        for batch_idx, (texts, mels) in enumerate(pbar):
            texts, mels = texts.to(device), mels.to(device)

            optimizer.zero_grad()

            # Прогон через модель
            pred_mels = model(texts)  # Выход (batch, time_steps, n_mels)
            
            # Транспонируем для корректного сопоставления размерностей
            pred_mels = pred_mels.permute(0, 2, 1)  # (batch, n_mels, time_steps)

            # Функция потерь (например, MSE)
            loss = criterion(pred_mels, mels)
            loss.backward()

            optimizer.step()
            total_loss += loss.item()

            # Обновляем описание прогресс-бара
            pbar.set_postfix(loss=loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / len(dataloader):.4f}")


Epoch 1/1000: 100%|██████████| 74/74 [00:01<00:00, 60.18batch/s, loss=2.51]


Epoch [1/1000], Average Loss: 7.0109


Epoch 2/1000: 100%|██████████| 74/74 [00:01<00:00, 60.08batch/s, loss=2.52]


Epoch [2/1000], Average Loss: 7.0020


Epoch 3/1000: 100%|██████████| 74/74 [00:01<00:00, 61.38batch/s, loss=2.53]


...

Epoch [998/1000], Average Loss: 6.3512


Epoch 999/1000: 100%|██████████| 74/74 [00:01<00:00, 68.21batch/s, loss=2.37]


Epoch [999/1000], Average Loss: 6.3523


Epoch 1000/1000: 100%|██████████| 74/74 [00:01<00:00, 70.60batch/s, loss=2.38]

Epoch [1000/1000], Average Loss: 6.3513


In [153]:
import librosa

In [141]:
def mel_to_audio(mel_spec, sr=16000, n_fft=1024, hop_length=256, n_mels=80):
    # 1. Отмена логарифмирования
    stft = librosa.feature.inverse.mel_to_stft(
        np.exp(mel_spec) - 1e-6,
        sr=sr,
        n_fft=n_fft,
        fmin=50,
        fmax=8000
    )
    
    audio = librosa.griffinlim(
        stft,
        n_iter=100,
        hop_length=hop_length,
        win_length=n_fft
    )
    
    # 3. Нормализация (чтобы избежать клиппинга)
    audio = librosa.util.normalize(audio) * 0.9
    
    return audio

In [154]:
Audio(mel_to_audio(mel_spectrograms[0]), rate=16000)

In [155]:
def token_padding(token_ids: list, vocab: dict = tokenizer.text_to_ids_voc, max_len: int = 27, ):
    while len(token_ids) != max_len:
        token_ids.append(vocab.get('<pad>'))
    return token_ids


In [157]:
text = df['token_ids'][0]
print(text)
print(f'target mel-gram shape: {df["mel_spec"][0].shape}')
print(df['array'][0])

[5165, 3734, 5298, 3734, 2808, 3734, 932, 3734, 460, 3734, 5910, 3734, 4008, 3734, 4773, 3734, 1197, 6674, 6674, 6674, 6674, 6674, 6674, 6674, 6674, 6674, 6674]
target mel-gram shape: (80, 429)
[-1.0186341e-10  6.9121597e-11 -8.0035534e-11 ...  5.7599618e-07
  5.1849293e-07 -2.8502502e-08]


In [158]:
# Получаем предсказанные мел-спектрограммы от модели
model.eval()
with torch.no_grad():
    pred_mels = model(texts)  # Получаем предсказание из модели

In [160]:
pred_mels_np = pred_mels.detach().cpu().numpy()
print(pred_mels_np)
print(pred_mels_np.shape)
pred_mels_np_T = pred_mels_np.squeeze(0).T 
print(pred_mels_np_T.shape)

[[[-3.5824232   2.6434946   2.9420912  ... -4.844888   -5.4557567
   -7.5538077 ]
  [-3.5476766   2.5091236   2.8119643  ... -5.025845   -5.6564283
   -7.7170267 ]
  [-3.5129304   2.3747528   2.6818376  ... -5.206802   -5.8571005
   -7.8802457 ]
  ...
  [-0.02061843 -0.01244909 -0.00966225 ... -0.04213148 -0.04252345
   -0.05813059]
  [-0.02061843 -0.01244909 -0.00966225 ... -0.04213148 -0.04252345
   -0.05813059]
  [-0.02061843 -0.01244909 -0.00966225 ... -0.04213148 -0.04252345
   -0.05813059]]]
(1, 429, 80)
(80, 429)


In [161]:
y = mel_to_audio(pred_mels_np_T)
y

array([ 0.11674957,  0.10413872,  0.09611181, ...,  0.09145977,
       -0.09135156,  0.04673128], dtype=float32)

In [147]:
from IPython.display import Audio

In [162]:
Audio(y, rate=16000)

In [164]:
import soundfile as sf
sf.write('output5.wav', y, 16000)  # Транспонируем для soundfile

In [167]:
torch.save(model.state_dict(), 'tts_model_words1.pth')  # Только веса
# ИЛИ
torch.save(model, 'tts_model_full_word1.pt')  # Веса + архитектура

In [ ]:
# Для state_dict
model = TTSTransformer(vocab_size, d_model=256, n_mels=80)
model.load_state_dict(torch.load('tts_model.pth'))
model.eval()

# Для полной модели
model = torch.load('tts_model_full.pt')
model.eval()

In [150]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(y.reshape(-1, 1), df['array'][0].reshape(-1, 1))